In [1]:
import numpy as np
import torch
from torch.nn import Parameter
from torch.nn.modules.module import Module
import torch.nn.functional as F
import math
from torch import nn

/home/maicg/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square conv kernel
        # self.conv1 = nn.Conv2d(1, 6, 5)
        # self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(784, 300)  # 5x5 image dimension
        self.fc2 = nn.Linear(300, 100)
        self.fc3 = nn.Linear(100, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim=1)
        return x

model = LeNet().to(device=device)

In [20]:
# cai dat mot so hyperparemeter
# Define some const

BATCH_SIZE = 128
EPOCHS = 100
LEARNING_RATE = 0.001
USE_CUDA = True
SEED = 42
LOG_AFTER = 10 # How many batches to wait before logging training status
LOG_FILE = 'log_prunting.txt'
SENSITIVITY = 2 # Sensitivity value that is multiplied to layer's std in order to get threshold value

# Control Seed
torch.manual_seed(SEED)

# Select Device
use_cuda = USE_CUDA and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else 'cpu')

In [22]:
# Cai dat dataloader
# Create the dataset with MNIST

from torchvision import datasets, transforms

# Train loader
kwargs = {'num_workers': 5, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True, **kwargs)

# Test loader
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=False, **kwargs)

In [23]:
# dinh nghia optimizer
import torch.optim as optim

# Define optimizer with Adam function
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=0.0001)
initial_optimizer_state_dict = optimizer.state_dict()

In [24]:
# training mo hinh

from tqdm import tqdm 
# Define training function 

def train(model):
    model.train()
    for epoch in range(EPOCHS):
        pbar = tqdm(enumerate(train_loader), total=len(train_loader))
        for batch_idx, (data, target) in pbar:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()

            # zero-out all the gradients corresponding to the pruned connections
            for name, p in model.named_parameters():
                tensor = p.data.cpu().numpy()
                grad_tensor = p.grad.data.cpu().numpy()
                grad_tensor = np.where(tensor==0, 0, grad_tensor)
                p.grad.data = torch.from_numpy(grad_tensor).to(device)

            optimizer.step()
            if batch_idx % LOG_AFTER == 0:
                done = batch_idx * len(data)
                percentage = 100. * batch_idx / len(train_loader)
                pbar.set_description(f'Train Epoch: {epoch} [{done:5}/{len(train_loader.dataset)} ({percentage:3.0f}%)]  Loss: {loss.item():.6f}')
    return model

In [25]:
model = train(model)

Train Epoch: 0 [58880/60000 ( 98%)]  Loss: 0.165757: 100%|██████████| 469/469 [00:01<00:00, 271.60it/s]
Train Epoch: 1 [58880/60000 ( 98%)]  Loss: 0.077967: 100%|██████████| 469/469 [00:01<00:00, 296.10it/s]
Train Epoch: 2 [58880/60000 ( 98%)]  Loss: 0.239732: 100%|██████████| 469/469 [00:01<00:00, 296.09it/s]
Train Epoch: 3 [58880/60000 ( 98%)]  Loss: 0.054293: 100%|██████████| 469/469 [00:01<00:00, 285.10it/s]
Train Epoch: 4 [58880/60000 ( 98%)]  Loss: 0.013612: 100%|██████████| 469/469 [00:01<00:00, 292.28it/s]
Train Epoch: 5 [58880/60000 ( 98%)]  Loss: 0.056440: 100%|██████████| 469/469 [00:01<00:00, 252.28it/s]
Train Epoch: 6 [58880/60000 ( 98%)]  Loss: 0.052944: 100%|██████████| 469/469 [00:01<00:00, 289.32it/s]
Train Epoch: 7 [58880/60000 ( 98%)]  Loss: 0.012814: 100%|██████████| 469/469 [00:01<00:00, 294.21it/s]
Train Epoch: 8 [58880/60000 ( 98%)]  Loss: 0.008846: 100%|██████████| 469/469 [00:01<00:00, 275.62it/s]
Train Epoch: 9 [58880/60000 ( 98%)]  Loss: 0.018314: 100%|██████

In [26]:
# test mo hinh
from time import time 

def test(model):
    start_time = time()
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)
        accuracy = 100. * correct / len(test_loader.dataset)
        print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%). Total time = {time() - start_time}')
    
    return accuracy

In [27]:
accuracy = test(model)

Test set: Average loss: 0.0890, Accuracy: 9801/10000 (98.01%). Total time = 0.3305783271789551


In [45]:
def print_nonzeros(model):
    nonzero = total = 0
    for name, p in model.named_parameters():
        tensor = p.data.cpu().numpy()
        nz_count = np.count_nonzero(tensor)
        total_params = np.prod(tensor.shape)
        nonzero += nz_count
        total += total_params
        print(f'{name:20} | nonzeros = {nz_count:7} / {total_params:7} ({100 * nz_count / total_params:6.2f}%) | total_pruned = {total_params - nz_count :7} | shape = {tensor.shape}')
    print(f'alive: {nonzero}, pruned : {total - nonzero}, total: {total}, Compression rate : {total/nonzero:10.2f}x  ({100 * (total-nonzero) / total:6.2f}% pruned)')



In [29]:
print_nonzeros(model)

fc1.weight           | nonzeros =  235200 /  235200 (100.00%) | total_pruned =       0 | shape = (300, 784)
fc1.bias             | nonzeros =     300 /     300 (100.00%) | total_pruned =       0 | shape = (300,)
fc2.weight           | nonzeros =   30000 /   30000 (100.00%) | total_pruned =       0 | shape = (100, 300)
fc2.bias             | nonzeros =     100 /     100 (100.00%) | total_pruned =       0 | shape = (100,)
fc3.weight           | nonzeros =    1000 /    1000 (100.00%) | total_pruned =       0 | shape = (10, 100)
fc3.bias             | nonzeros =      10 /      10 (100.00%) | total_pruned =       0 | shape = (10,)
alive: 266610, pruned : 0, total: 266610, Compression rate :       1.00x  (  0.00% pruned)


In [33]:

# Customize of prune function with mask
def prune(module, threshold): #tinh toan lai cac trong so co weight nho hon nguong quy dinh, cap nhat lai mask va weight tai cac vi tri do ve gia tri 0
    weight_dev = module.weight.device
    mask_module =  Parameter(torch.ones([module.out_features, module.in_features]), requires_grad=False)
    mask_dev = mask_module.device 
    # Convert Tensors to numpy and calculate
    tensor = module.weight.data.cpu().numpy()
    mask = mask_module.data.cpu().numpy()
    new_mask = np.where(abs(tensor) < threshold, 0, mask)
    # Apply new weight and mask
    module.weight.data = torch.from_numpy(tensor * new_mask).to(weight_dev)
    mask_module = torch.from_numpy(new_mask).to(mask_dev)

def prune_by_std(module, s=0.25): # tuy chinh tham so s=0.25 de tinh toan gia tri cua threshold can cat tia, 25% của average standard deviation: gia tri do lech chuan trung binh
    # Note that module here is the layer
    # ex) fc1, fc2, fc3
    threshold = np.std(module.weight.data.cpu().numpy()) * s
    print(f'Pruning with threshold : {threshold} for layer')
    prune(module, threshold)

In [34]:
prune_by_std(model.fc1)
prune_by_std(model.fc2)
prune_by_std(model.fc3)

Pruning with threshold : 0.009447183459997177 for layer
Pruning with threshold : 0.014730214141309261 for layer
Pruning with threshold : 0.04688509926199913 for layer


In [35]:
accuracy = test(model)

Test set: Average loss: 0.0971, Accuracy: 9783/10000 (97.83%). Total time = 0.31531476974487305


In [36]:
print_nonzeros(model)

fc1.weight           | nonzeros =   97114 /  235200 ( 41.29%) | total_pruned =  138086 | shape = (300, 784)
fc1.bias             | nonzeros =     300 /     300 (100.00%) | total_pruned =       0 | shape = (300,)
fc2.weight           | nonzeros =   13315 /   30000 ( 44.38%) | total_pruned =   16685 | shape = (100, 300)
fc2.bias             | nonzeros =     100 /     100 (100.00%) | total_pruned =       0 | shape = (100,)
fc3.weight           | nonzeros =     723 /    1000 ( 72.30%) | total_pruned =     277 | shape = (10, 100)
fc3.bias             | nonzeros =      10 /      10 (100.00%) | total_pruned =       0 | shape = (10,)
alive: 111562, pruned : 155048, total: 266610, Compression rate :       2.39x  ( 58.16% pruned)


In [37]:
# Retraining
optimizer.load_state_dict(initial_optimizer_state_dict) # Reset the optimizer

model = train(model)

Train Epoch: 0 [58880/60000 ( 98%)]  Loss: 0.000863: 100%|██████████| 469/469 [00:01<00:00, 263.28it/s]
Train Epoch: 1 [58880/60000 ( 98%)]  Loss: 0.002972: 100%|██████████| 469/469 [00:01<00:00, 279.88it/s]
Train Epoch: 2 [58880/60000 ( 98%)]  Loss: 0.008068: 100%|██████████| 469/469 [00:01<00:00, 254.22it/s]
Train Epoch: 3 [58880/60000 ( 98%)]  Loss: 0.003958: 100%|██████████| 469/469 [00:02<00:00, 219.82it/s]
Train Epoch: 4 [58880/60000 ( 98%)]  Loss: 0.000260: 100%|██████████| 469/469 [00:01<00:00, 288.83it/s]
Train Epoch: 5 [58880/60000 ( 98%)]  Loss: 0.002234: 100%|██████████| 469/469 [00:01<00:00, 271.66it/s]
Train Epoch: 6 [58880/60000 ( 98%)]  Loss: 0.000692: 100%|██████████| 469/469 [00:01<00:00, 240.74it/s]
Train Epoch: 7 [58880/60000 ( 98%)]  Loss: 0.000634: 100%|██████████| 469/469 [00:01<00:00, 250.36it/s]
Train Epoch: 8 [58880/60000 ( 98%)]  Loss: 0.005515: 100%|██████████| 469/469 [00:01<00:00, 253.42it/s]
Train Epoch: 9 [58880/60000 ( 98%)]  Loss: 0.002394: 100%|██████

In [38]:
accuracy = test(model)

Test set: Average loss: 0.0729, Accuracy: 9828/10000 (98.28%). Total time = 0.34559106826782227


In [39]:
print_nonzeros(model)

fc1.weight           | nonzeros =   97114 /  235200 ( 41.29%) | total_pruned =  138086 | shape = (300, 784)
fc1.bias             | nonzeros =     300 /     300 (100.00%) | total_pruned =       0 | shape = (300,)
fc2.weight           | nonzeros =   13315 /   30000 ( 44.38%) | total_pruned =   16685 | shape = (100, 300)
fc2.bias             | nonzeros =     100 /     100 (100.00%) | total_pruned =       0 | shape = (100,)
fc3.weight           | nonzeros =     723 /    1000 ( 72.30%) | total_pruned =     277 | shape = (10, 100)
fc3.bias             | nonzeros =      10 /      10 (100.00%) | total_pruned =       0 | shape = (10,)
alive: 111562, pruned : 155048, total: 266610, Compression rate :       2.39x  ( 58.16% pruned)


In [41]:
# LUONG TU HOA VA SHARE WEIGHT
from sklearn.cluster import KMeans
from scipy.sparse import csc_matrix, csr_matrix #hai format de luu tru ma tran thu nham tinh toan duoc de dang do tiet kiem bo nho.

def apply_weight_sharing(model, bits=5): # co toi da 2^5 = 32 cum cua kmeans
    for module in model.children():
        dev = module.weight.device
        weight = module.weight.data.cpu().numpy()
        shape = weight.shape
        mat = csr_matrix(weight) if shape[0] < shape[1] else csc_matrix(weight)
        min_ = min(mat.data)
        max_ = max(mat.data)
        space = np.linspace(min_, max_, num=2**bits)
        kmeans = KMeans(n_clusters=len(space), init=space.reshape(-1,1), n_init=1, algorithm="full")
        kmeans.fit(mat.data.reshape(-1,1))
        new_weight = kmeans.cluster_centers_[kmeans.labels_].reshape(-1) # share lại centroid vào các vị trí weight
        mat.data = new_weight
        module.weight.data = torch.from_numpy(mat.toarray()).to(dev)

    return model

In [42]:
apply_weight_sharing(model)

/home/maicg/anaconda3/envs/Face-insight/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1318: FutureWarning: algorithm='full' is deprecated, it will be removed in 1.3. Using 'lloyd' instead.
  warnings.warn(
/home/maicg/anaconda3/envs/Face-insight/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1318: FutureWarning: algorithm='full' is deprecated, it will be removed in 1.3. Using 'lloyd' instead.
  warnings.warn(
/home/maicg/anaconda3/envs/Face-insight/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1318: FutureWarning: algorithm='full' is deprecated, it will be removed in 1.3. Using 'lloyd' instead.
  warnings.warn(


LeNet(
  (fc1): Linear(in_features=784, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=10, bias=True)
)

In [43]:
accuracy = test(model)

Test set: Average loss: 0.0714, Accuracy: 9831/10000 (98.31%). Total time = 0.36731457710266113


In [48]:
print_nonzeros(model)

fc1.weight           | nonzeros =   97114 /  235200 ( 41.29%) | total_pruned =  138086 | shape = (300, 784)
fc1.bias             | nonzeros =     300 /     300 (100.00%) | total_pruned =       0 | shape = (300,)
fc2.weight           | nonzeros =   13315 /   30000 ( 44.38%) | total_pruned =   16685 | shape = (100, 300)
fc2.bias             | nonzeros =     100 /     100 (100.00%) | total_pruned =       0 | shape = (100,)
fc3.weight           | nonzeros =     723 /    1000 ( 72.30%) | total_pruned =     277 | shape = (10, 100)
fc3.bias             | nonzeros =      10 /      10 (100.00%) | total_pruned =       0 | shape = (10,)
alive: 111562, pruned : 155048, total: 266610, Compression rate :       2.39x  ( 58.16% pruned)
